# **Task: To predict whether the user likes the mobile phone or not.**

**Assumption:** If the **average rating**** of mobile **>= *threshold***, then the user likes it, otherwise not.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, log_loss
from random import randint
import operator
import json
from IPython import display
import os
import warnings

np.random.seed(0)
warnings.filterwarnings("ignore")
THRESHOLD = 8

In [ ]:
train = pd.read_csv('/kaggle/input/padhai-mp-neuron-like-unlike-classification/train.csv', skipinitialspace = True)
train.shape

In [ ]:
test = pd.read_csv('/kaggle/input/padhai-mp-neuron-like-unlike-classification/test.csv', skipinitialspace = True)
test.shape

**Missing values:**

'Also Known As'(459),'Applications'(421),'Audio Features'(437),'Bezel-less display'(266),'Browser'(449),'Build Material'(338),'Co-Processor'(451),'Display Colour'(457),'Mobile High-Definition Link(MHL)'(472),'Music'(447) 'Email','Fingerprint Sensor Position'(174),'Games'(446),'HDMI'(454),'Heart Rate Monitor'(467),'IRIS Scanner'(467), 'Optical Image Stabilisation'(219),'Other Facilities'(444),'Phone Book'(444),'Physical Aperture'(87),'Quick Charging'(122),'Ring Tone'(444),'Ruggedness'(430),SAR Value(315),'SIM 3'(472),'SMS'(470)', 'Screen Protection'(229),'Screen to Body Ratio (claimed by the brand)'(428),'Sensor'(242),'Software Based Aperture'(473), 'Special Features'(459),'Standby time'(334),'Stylus'(473),'TalkTime'(259), 'USB Type-C'(374),'Video Player'(456), 'Video Recording Features'(458),'Waterproof'(398),'Wireless Charging','USB OTG Support'(159), 'Video ,'Recording'(113),'Java'(471),'Browser'(448)


**Very low variance:**

'Architecture'(most entries are 64-bit),'Audio Jack','GPS','Loudspeaker','Network','Network Support','Other Sensors'(28),'SIM Size', 'VoLTE'


**Multivalued:**

'Colours','Custom UI','Model'(1),'Other Sensors','Launch Date'


**Not important:**

'Bluetooth', 'Settings'(75),'Wi-Fi','Wi-Fi Features'


**Doubtful:**

'Aspect Ratio','Autofocus','Brand','Camera Features','Fingerprint Sensor'(very few entries are missing), 'Fingerprint Sensor Position', 'Graphics'(multivalued),'Image resolution'(multivalued),'SIM Size','Sim Slot(s)', 'User Available Storage', 'SIM 1', 'SIM 2','Shooting Modes', 'Touch Screen'(24), 'USB Connectivity'


**To check:**

'Display Type','Expandable Memory','FM Radio'


**High Correlation with other features**

'SIM Slot(s)' high correlation with SIM1 'Weight' has high high correlation with capacity , screen-to-body ratio 'Height' - screen size is also there


**Given a mobile, we can't directly get these features**

'Rating Count', 'Review Count'


**Keeping:**

'Capacity','Flash'(17),'Height'(22),'Internal Memory'(20, require cleaning),'Operating System'(25, require cleaning), 'Pixel Density'(1, clean it),'Processor'(22, clean it), 'RAM'(17, clean), 'Rating','Resolution'(cleaning), 'Screen Resolution','Screen Size', 'Thickness'(22), 'Type','User Replaceable','Weight'(cleaning),'Sim Size'(), 'Other Sensors'(28), 'Screen to Body Ratio (calculated)','Width'

In [ ]:
def data_clean(data):
    
    # Let's first remove all missing value features
    columns_to_remove = ['Also Known As','Applications','Audio Features','Bezel-less display'
                         'Browser','Build Material','Co-Processor','Browser'
                         'Display Colour','Mobile High-Definition Link(MHL)',
                         'Music', 'Email','Fingerprint Sensor Position',
                         'Games','HDMI','Heart Rate Monitor','IRIS Scanner', 
                         'Optical Image Stabilisation','Other Facilities',
                         'Phone Book','Physical Aperture','Quick Charging',
                         'Ring Tone','Ruggedness','SAR Value','SIM 3','SMS',
                         'Screen Protection','Screen to Body Ratio (claimed by the brand)',
                         'Sensor','Software Based Aperture', 'Special Features',
                         'Standby time','Stylus','TalkTime', 'USB Type-C',
                         'Video Player', 'Video Recording Features','Waterproof',
                         'Wireless Charging','USB OTG Support', 'Video Recording','Java']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    #Features having very low variance 
    columns_to_remove = ['Architecture','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Multivalued:
    columns_to_remove = ['Architecture','Launch Date','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE', 'Custom UI']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Not much important
    columns_to_remove = ['Bluetooth', 'Settings','Wi-Fi','Wi-Fi Features']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    
    return data

In [ ]:
train = data_clean(train)
test = data_clean(test)

In [ ]:
print(train.shape)
print(test.shape)

### **Removing all those data points in which more than 15 features are missing**

In [ ]:
train = train[(train.isnull().sum(axis=1) <= 15)]

# You shouldn't remove data points from test set
#test = test[(test.isnull().sum(axis=1) <= 15)]

In [ ]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

#### **Filling missing values**

In [ ]:
def for_integer(test):
    try:
        test = test.strip()
        return int(test.split(' ')[0])
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass

def for_string(test):
    try:
        test = test.strip()
        return (test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

def for_float(test):
    try:
        test = test.strip()
        return float(test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

    
def for_Internal_Memory(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[1] == 'GB':
            return int(test[0])
        if test[1] == 'MB':
#             print("here")
            return (int(test[0]) * 0.001)
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass
    
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

In [ ]:
def data_clean_2(x):
    data = x.copy()
    
    data['Capacity'] = data['Capacity'].apply(for_integer)

    data['Height'] = data['Height'].apply(for_float)
    data['Height'] = data['Height'].fillna(data['Height'].mean())

    data['Internal Memory'] = data['Internal Memory'].apply(for_Internal_Memory)

    data['Pixel Density'] = data['Pixel Density'].apply(for_integer)

    data['Internal Memory'] = data['Internal Memory'].fillna(data['Internal Memory'].median())
    data['Internal Memory'] = data['Internal Memory'].astype(int)

    data['RAM'] = data['RAM'].apply(for_integer)
    data['RAM'] = data['RAM'].fillna(data['RAM'].median())
    data['RAM'] = data['RAM'].astype(int)

    data['Resolution'] = data['Resolution'].apply(for_integer)
    data['Resolution'] = data['Resolution'].fillna(data['Resolution'].median())
    data['Resolution'] = data['Resolution'].astype(int)

    data['Screen Size'] = data['Screen Size'].apply(for_float)

    data['Thickness'] = data['Thickness'].apply(for_float)
    data['Thickness'] = data['Thickness'].fillna(data['Thickness'].mean())
    data['Thickness'] = data['Thickness'].round(2)

    data['Type'] = data['Type'].fillna('Li-Polymer')

    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].apply(for_float)
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].fillna(data['Screen to Body Ratio (calculated)'].mean())
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].round(2)

    data['Width'] = data['Width'].apply(for_float)
    data['Width'] = data['Width'].fillna(data['Width'].mean())
    data['Width'] = data['Width'].round(2)

    data['Flash'][data['Flash'].isna() == True] = "Other"

    data['User Replaceable'][data['User Replaceable'].isna() == True] = "Other"

    data['Num_cores'] = data['Processor'].apply(for_string)
    data['Num_cores'][data['Num_cores'].isna() == True] = "Other"


    data['Processor_frequency'] = data['Processor'].apply(find_freq)
    #because there is one entry with 208MHz values, to convert it to GHz
    data['Processor_frequency'][data['Processor_frequency'] > 200] = 0.208
    data['Processor_frequency'] = data['Processor_frequency'].fillna(data['Processor_frequency'].mean())
    data['Processor_frequency'] = data['Processor_frequency'].round(2)

    data['Camera Features'][data['Camera Features'].isna() == True] = "Other"

    #simplifyig Operating System to os_name for simplicity
    data['os_name'] = data['Operating System'].apply(for_string)
    data['os_name'][data['os_name'].isna() == True] = "Other"

    data['Sim1'] = data['SIM 1'].apply(for_string)

    data['SIM Size'][data['SIM Size'].isna() == True] = "Other"

    data['Image Resolution'][data['Image Resolution'].isna() == True] = "Other"

    data['Fingerprint Sensor'][data['Fingerprint Sensor'].isna() == True] = "Other"

    data['Expandable Memory'][data['Expandable Memory'].isna() == True] = "No"

    data['Weight'] = data['Weight'].apply(for_integer)
    data['Weight'] = data['Weight'].fillna(data['Weight'].mean())
    data['Weight'] = data['Weight'].astype(int)

    data['SIM 2'] = data['SIM 2'].apply(for_string)
    data['SIM 2'][data['SIM 2'].isna() == True] = "Other"
    
    return data

In [ ]:
train = data_clean_2(train)
test = data_clean_2(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

**Not very important feature**

In [ ]:
def data_clean_3(x):
    
    data = x.copy()

    columns_to_remove = ['User Available Storage','SIM Size','Chipset','Processor','Autofocus','Aspect Ratio','Touch Screen',
                        'Bezel-less display','Operating System','SIM 1','USB Connectivity','Other Sensors','Graphics','FM Radio',
                        'NFC','Shooting Modes','Browser','Display Colour' ]

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = [ 'Screen Resolution','User Replaceable','Camera Features',
                        'Thickness', 'Display Type']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = ['Fingerprint Sensor', 'Flash', 'Rating Count', 'Review Count','Image Resolution','Type','Expandable Memory',\
                        'Colours','Width','Model']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    return data

In [ ]:
train = data_clean_3(train)
test = data_clean_3(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

In [ ]:
# one hot encoding

train_ids = train['PhoneId']
test_ids = test['PhoneId']

cols = list(test.columns)
cols.remove('PhoneId')
cols.insert(0, 'PhoneId')

combined = pd.concat([train.drop('Rating', axis=1)[cols], test[cols]])
print(combined.shape)
print(combined.columns)

combined = pd.get_dummies(combined)
print(combined.shape)
print(combined.columns)

train_new = combined[combined['PhoneId'].isin(train_ids)]
test_new = combined[combined['PhoneId'].isin(test_ids)]

In [ ]:
train_new = train_new.merge(train[['PhoneId', 'Rating']], on='PhoneId')

In [ ]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train_new.shape[0])
print("Number of features in train: %d" % train_new.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test_new.shape[0])
print("Number of features in test: %d" % test_new.shape[1])

In [ ]:
train_new.head()

In [ ]:
test_new.head()

### **Normalize data points so that we get binarize effectively**

In [ ]:
scalar = StandardScaler()

In [ ]:
#train_new[['Screen to Body Ratio (calculated)', 'Screen Size', 'Capacity', 'RAM', 'Resolution', 'Pixel Density', 'Height', 'Internal Memory', 'Weight', 'Processor_frequency']] =\
#train_new[['Screen to Body Ratio (calculated)', 'Screen Size', 'Capacity', 'RAM', 'Resolution', 'Pixel Density', 'Height', 'Internal Memory', 'Weight', 'Processor_frequency']].\
#apply(lambda x: StandardScaler().fit_transform(x).to_numpy())

In [ ]:
train_new[['Screen to Body Ratio (calculated)', 'Screen Size', 'Capacity', 'RAM', 'Resolution', 'Pixel Density', 'Height', 'Internal Memory', 'Weight', 'Processor_frequency']] = \
scalar.fit_transform(train_new[['Screen to Body Ratio (calculated)', 'Screen Size', 'Capacity', 'RAM', 'Resolution', 'Pixel Density', 'Height', 'Internal Memory', 'Weight', 'Processor_frequency']].values)

In [ ]:
train_new.head()

#### **Binarization of input dataframe to send it into MP Neuron model**

In [ ]:
train_new.set_index('PhoneId', inplace = True)

In [ ]:
train_new[['Screen to Body Ratio (calculated)', 'Screen Size', 'Capacity', 'RAM', 'Resolution', 'Pixel Density', 'Height', 'Internal Memory', 'Weight', 'Processor_frequency']] = train_new[['Screen to Body Ratio (calculated)', 'Screen Size', 'Capacity', 'RAM', 'Resolution', 'Pixel Density', 'Height', 'Internal Memory', 'Weight', 'Processor_frequency']].apply(pd.cut, bins=2, labels=[1,0])

In [ ]:
train_new.head()

### **MP NEURON**

In [ ]:
X = train_new.drop('Rating', axis = 1)
X.head()

In [ ]:
X[['Screen to Body Ratio (calculated)', 'Screen Size', 'Capacity', 'RAM', 'Resolution', 'Pixel Density', 'Height', 'Internal Memory', 'Weight', 'Processor_frequency']] = train_new[['Screen to Body Ratio (calculated)', 'Screen Size', 'Capacity', 'RAM', 'Resolution', 'Pixel Density', 'Height', 'Internal Memory', 'Weight', 'Processor_frequency']] = train_new[['Screen to Body Ratio (calculated)', 'Screen Size', 'Capacity', 'RAM', 'Resolution', 'Pixel Density', 'Height', 'Internal Memory', 'Weight', 'Processor_frequency']] = X[['Screen to Body Ratio (calculated)', 'Screen Size', 'Capacity', 'RAM', 'Resolution', 'Pixel Density', 'Height', 'Internal Memory', 'Weight', 'Processor_frequency']].astype('uint8')

In [ ]:
y = train_new.Rating

In [ ]:
# i = randint(0, X.shape[0])

# print('For row - ', i)

# if (np.sum(X).all() >= THRESHOLD):
#     print('MP Neuron inference is Malignant')
# else:
#     print('MP Neuron inference is benign')
    
# if (y[i] >= THRESHOLD):
#   print('Ground truth is malignant')
# else:
#   print('Ground truth is benign')

In [ ]:
# # Predicting y from model fitted to X_train and y_train

# ypred_train = []
# accurate_rows = 0

# for i, j in zip(X, y):
#   ypred = (np.sum(i, axis = 1) >= THRESHOLD)
#   ypred_train.append(ypred)
#   accurate_rows += (j==ypred)
  
  
#print(accurate_rows, accurate_rows/X.shape[0])

In [ ]:
# for b in range(0, X_binarised_train.shape[1] + 1):
#   Y_pred_train = []
#   accurate_rows = 0
  
#   for x, y in zip(X_binarised_train, Y_train):
#     y_pred = (np.sum(x) >= b)
#     Y_pred_train.append(y_pred)
#     accurate_rows += (y == y_pred)
    
#   print(b, accurate_rows/X_binarised_train.shape[0]) 

In [ ]:
# # Let's check with a single value of b

# b = 28

# Y_pred_test = []

# for x in X_binarised_test:
#   y_pred = (np.sum(x) >= b)
#   Y_pred_test.append(y_pred)

# accuracy = accuracy_score(Y_pred_test, Y_test)

# print(b, accuracy) 

In [ ]:
class MPNeuron:
    THRESHOLD = 10
    
    def __init__(self):
        self.b = THRESHOLD
        
    def model(self, x):
        return (np.sum(x, axis = 1).values >= self.b)
    
    def predict(self, X):
        Y = []
        for x in X:
            result = self.model(x)
            Y.append(result)
        return np.array(Y)
    
    def fit(self, X, Y):
        accuracy = {}
        
        for b in range(X.shape[1] + 1):
            self.b = b
            Y_pred = self.predict(X)
            accuracy[b] = accuracy_score(Y_pred, Y)
            
        best_b = max(accuracy, key = accuracy.get)
        self.b = best_b
        
        print('Optimal value of b is', best_b)
        print('Highest accuracy is', accuracy[best_b])

In [ ]:
neuro = MPNeuron()
neuro_fit = neuro.fit(X, y)